In [30]:
import imaplib
import getpass
import email
import re
from PyPDF4 import PdfFileReader
import io

IMAP_SERVER = 'imap.gmail.com' 
IMAP_PORT = 993
FROM_EMAIL = "digitalerkassenzettel@coop.ch"
USER, PASSWORD = 'kostik.semenov@gmail.com','ryzsak-jigxaM-zohsy1'


list_response_pattern = re.compile(
    r'\((?P<flags>.*?)\) "(?P<delimiter>.*)" (?P<name>.*)'
)


def parse_list_response(line):
    match = list_response_pattern.match(line.decode('utf-8'))
    flags, delimiter, mailbox_name = match.groups()
    mailbox_name = mailbox_name.strip('"')
    return (flags, delimiter, mailbox_name)

def mailbox_connect(host, port, user = None, password = None):
    if user is None:
        user = getpass.getuser("Username:")
    if password is None:
        password = getpass.getpass("Password:")
    mail_connection = None
    mail_connection = imaplib.IMAP4_SSL(host, port=port)
    mail_connection.login(user, password)
    return mail_connection

def get_all_mailbox_name(connection):
    box_name = None
    mailboxes = connection.list()
    for box in mailboxes[1]:
        flag, delim, name = parse_list_response(box)
        if "\\All" in flag.split():
            box_name = name
    return box_name

def get_message(message_num, mail_connection):
    email_message = None
    result, message_data = mail_connection.uid('fetch', message_num, '(RFC822)')
    try:
        email_message = email.message_from_string(message_data[0][1])   
    except TypeError:
        email_message = email.message_from_bytes(message_data[0][1])
    return email_message

connection = mailbox_connect(IMAP_SERVER, IMAP_PORT, USER, PASSWORD)
reciept_list = []

box_name = get_all_mailbox_name(connection)
connection.select(f'"{box_name}"')
result, data = connection.uid('search', None, f"(FROM '{FROM_EMAIL}')")
if result == 'OK':
    for num in data[0].split():
        message = get_message(num, connection)
        
        for element in message.walk():
            if element.get_content_type().startswith("application"):
                reciept_list.append((element.get_filename(),element.get_payload(decode=True)))
    
if len(reciept_list) > 0:
    file_name, file_data = reciept_list[0]
    pdf_data = PdfFileReader(io.BytesIO(file_data))
    for page_num in range(0, pdf_data.getNumPages()):
        pprint.pprint(pdf_data.getPage(page_num).extractText())

    #print(pdf_data)
    
connection.logout()

NameError: name 'pprint' is not defined

In [29]:
# coding=utf8
# the above tag defines encoding for this document and is for Python 2.x compatibility

import re



test_str = ("Zug Neustadt\n"
	"02.03.20\n"
	"09:23\n"
	"02032\n"
	"00153616\n"
	"002\n"
	"0001921\n"
	"Artikel\n"
	"Menge\n"
	"Preis\n"
	"Aktion\n"
	"Total\n"
	"Zusatz\n"
	"Halbweissbrot 500G\n"
	"1\n"
	"1.20\n"
	"1.20\n"
	"0\n"
	"Danone Actimel Erdbeere 10x100g\n"
	"1\n"
	"7.95\n"
	"7.95\n"
	"0\n"
	"Coop Vollmilch past. Tetra Top ESL 1L\n"
	"1\n"
	"1.60\n"
	"1.60\n"
	"0\n"
	"Betty Bossi MH Orangensaft 750ML\n"
	"1\n"
	"3.30\n"
	"3.30\n"
	"0\n"
	"Coop Vollmilch past. Tetra Top ESL 1L\n"
	"1\n"
	"1.60\n"
	"1.60\n"
	"0\n"
	"Coop Wienerli Hello Family 2 Paar\n"
	"1\n"
	"3.35\n"
	"3.35\n"
	"0\n"
	"Coop Wienerli Hello Family 2 Paar\n"
	"1\n"
	"3.35\n"
	"3.35\n"
	"0\n"
	"Finish All-in-1 Edition Alpine 35 Tabs\n"
	"1\n"
	"16.50\n"
	"16.50\n"
	"Total CHF\n"
	"38.85\n"
	"VISA\n"
	"38.85\n"
	"Buchung\n"
	"XXXXXX******1732\n"
	"02.03.2020\n"
	"#31413264*00280775/092324#\n"
	"Total-EFT CHF:\n"
	"VISA\n"
	"00:00\n"
	"38.85\n"
	"COOP GENOSSENSCHAFT, CHE-116.311.185 MWST\n"
	"GR\n"
	"MWST%\n"
	"TOTAL\n"
	"MWST\n"
	"1\n"
	"7.70\n"
	"16.50\n"
	"1.18\n"
	"0\n"
	"2.50\n"
	"22.35\n"
	"0.55\n"
	"SUPERCARD NR.:  2501*****6333\n"
	"ALTER PUNKTESTAND\n"
	"SALDO\n"
	"34\n"
	"SUB-/TOTAL EXCL*\n"
	"38\n"
	"SUPERPUNKTE AUS EINKAUF\n"
	"38\n"
	"NEUER PUNKTESTAND\n"
	"SALDO\n"
	"72\n"
	"Seite 1/\n"
	"2\n\n\n"
	"Vielen Dank für Ihren Einkauf\n"
	"02.03.20\n"
	"09:23\n"
	"02032\n"
	"00153616\n"
	"002\n"
	"0001921\n"
	"Seite 2/\n"
	"2\n")

wordslist = test_str.split("\n")
a = wordslist.index('Zusatz')+1
b = wordslist.index('Total CHF')
wordslist[a:b]

['Halbweissbrot 500G',
 '1',
 '1.20',
 '1.20',
 '0',
 'Danone Actimel Erdbeere 10x100g',
 '1',
 '7.95',
 '7.95',
 '0',
 'Coop Vollmilch past. Tetra Top ESL 1L',
 '1',
 '1.60',
 '1.60',
 '0',
 'Betty Bossi MH Orangensaft 750ML',
 '1',
 '3.30',
 '3.30',
 '0',
 'Coop Vollmilch past. Tetra Top ESL 1L',
 '1',
 '1.60',
 '1.60',
 '0',
 'Coop Wienerli Hello Family 2 Paar',
 '1',
 '3.35',
 '3.35',
 '0',
 'Coop Wienerli Hello Family 2 Paar',
 '1',
 '3.35',
 '3.35',
 '0',
 'Finish All-in-1 Edition Alpine 35 Tabs',
 '1',
 '16.50',
 '16.50']

In [112]:
   

#getpass.getpass()
mail_connection = imaplib.IMAP4_SSL('imap.gmail.com', port=993)
mail_connection.login(b'kostik.semenov@gmail.com','ryzsak-jigxaM-zohsy1')
mailboxes = mail_connection.list()
all_mailbox = ""
for box in mailboxes[1]:
    flag, delim, name = parse_list_response(box)
    if "\\All" in flag.split():
        all_mailbox = name
        break
  
mail_connection.select(f'"{all_mailbox}"')

reciept_list=[]
result, data = mail_connection.uid('search', None, "(FROM 'digitalerkassenzettel@coop.ch')")
for num in data[0].split():
    result, message_data= mail_connection.uid('fetch', num, '(RFC822)')
    try:
        email_message = email.message_from_string(message_data[0][1])   
    except TypeError:
        email_message = email.message_from_bytes(message_data[0][1])
    
    for element in email_message.walk():
        content_type = element.get_content_type()
        if content_type.startswith("application"):
            reciept_list.append((element.get_filename(),element.get_payload(decode=True)))
    
    
mail_connection.logout()

KeyboardInterrupt: 